In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#importing required library,packages

import networkx as nx
%matplotlib inline
import matplotlib


import pandas as pd
import numpy as np

from __future__ import print_function
from matplotlib import pyplot as plt
import seaborn as sns

from scipy import stats

from imblearn.over_sampling import SMOTE

import glob

import sklearn
from sklearn import preprocessing



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [ ]:
#this file is obtained after running MHEALTH_save_data.ipynb
filename='/content/drive/My Drive/MHEALTHDATASET/rawdata.csv'

df=pd.read_csv(filename)
df=df.drop(['Unnamed: 0'],axis=1)
df['encoded-activity']=df['encoded-activity']-1
print("\n... dataset read---\n",df)

        feature-1  feature-2  ...  user-id         activity
0         -9.7788    0.55690  ...        1   Standing still
1         -9.7733    0.27880  ...        1   Standing still
2         -9.8609    0.11561  ...        1   Standing still
3         -9.7409    0.17652  ...        1   Standing still
4         -9.7821    0.21637  ...        1   Standing still
...           ...        ...  ...      ...              ...
343190    -8.6500   -0.83185  ...        9  Climbing stairs
343191    -8.9892   -1.04290  ...        9  Climbing stairs
343192    -9.4536   -1.05650  ...        9  Climbing stairs
343193   -10.1930   -1.43780  ...        9  Climbing stairs
343194   -10.1070   -1.72800  ...        9  Climbing stairs

[343195 rows x 26 columns]


In [ ]:
 
TIME_STEPS=128
FEATURES=23
NUM_CLASSES=12

In [ ]:
#defining function for normalizing data -column
def feature_normalize(arr):
  arr=arr.astype(float)
  mu=np.mean(arr)
  sigma=np.std(arr)
  return (arr-mu)/sigma

In [ ]:
#defining function to return segment and label
def segment_labeling_overall(dataframe,time_step,steps,features):
  segments=[]
  labels=[]
  node_names=[]
  n_features=3
  cnt=1
  r=(len(dataframe))//time_step+1
  print("Maximum no=",r)
  for j in range(FEATURES):
    dataframe['feature-'+str(j+1)]=feature_normalize(dataframe['feature-'+str(j+1)])
    
  for i in range(0,len(dataframe)-time_step,steps):
    feature_row=[]
    for j in range(FEATURES):
      feature_col=dataframe['feature-'+str(j+1)].values[i:i+time_step]
      mean_feature=np.mean(feature_col)
      feature_row.append(mean_feature)


    label=stats.mode(dataframe['encoded-activity'].values[i:i+time_step])[0][0]
    node_name=stats.mode(dataframe['user-id'].values[i:i+time_step])[0][0]
    segments.append(feature_row)
    labels.append(label)
    node_names.append(node_name)
    cnt+=1
    
  print("Done-->total"+str(cnt)+"nodes formed")
  segments=np.asarray(segments)
  labels=np.asarray(labels)
  node_names=np.asarray(node_names)
  return segments,labels,node_names

In [ ]:
node_id=1
graph_id=1

In [ ]:
node_map={}
graph_id_map={}
segments_map={}
list_of_nodes=[]
list_of_edges=[]
attr_list=[]
node_label_list=[]
graph_label_list=[]

In [ ]:
#creating one graph for given segments
def create_graph(labels,segments,colors):
  global graph_id,node_id,node_map,graph_id_map,segments_map,list_of_nodes,list_of_edges,attr_list,node_label_list
  node_colors=[]
  edge_colors=[]
  edge_list=[]
  node_list=[]
  vertices_by_activity=[]
  edges_by_color=[]
  black_edges=[]
  for i in range (NUM_CLASSES):
    empty_list=list()
    vertices_by_activity.append(empty_list)

  for i in range (NUM_CLASSES):
    empty_list=list()
    edges_by_color.append(empty_list)

  #print(vertices_by_activity)
  no_of_nodes=labels.shape[0]
  print("Total nodes present in the graph are-->"+str(no_of_nodes))

  for i in range (no_of_nodes):
    print("Label:")
    label=labels[i]
    
    print("Label of node-->"+str(label))

    node_colors.append(colors[label])
    attr_list.append(segments)
    node_label_list.append(label)
    
    node_name=str(graph_id)+'-'+colors[label]+':'+str(i)

    list_of_nodes.append(node_name)
    node_list.append(node_name)
    #print(vertices_by_activity[label])
    vertices_by_activity[label].append(node_name)
    node_map[node_name]=node_id
    segments_map[node_name]=segments[i]
    node_id+=1
  
  for i in range (NUM_CLASSES):
    list_=vertices_by_activity[i]
    
    if(len(list_)>0):
      for k in range(len(list_)-1):
        node1=list_[k] 
        node2=list_[k+1]
        
        edge=(node1,node2)
        edge_list.append(edge)
        edges_by_color[i].append((node1,node2))
        list_of_edges.append(edge)
        edge_colors.append(colors[i])
  
  for i in range (NUM_CLASSES):
    list1=vertices_by_activity[i]
    if(len(list1)>0):
      node1=list1[0]
      for j in range (i):
        list2=vertices_by_activity[j]
        if(len(list2)>0):
          node2=list2[0]
          edge=(node1,node2)
          edge_list.append(edge)
          black_edges.append((node1,node2))
          list_of_edges.append(edge)
          
  
  
  

  G=nx.Graph()
  G.add_nodes_from(node_list)
  G.add_edges_from(edge_list)
  print("Nodes of the graphs are: ",G.nodes())
  print("Edges of the graphs are: ",G.edges())
  
  
  pos=nx.spring_layout(G)
  
  plt.figure(figsize=(15,15))

  for i in range(NUM_CLASSES):
    colored_node_list=[]
    vi_list=vertices_by_activity[i]
    for v in vi_list:
      colored_node_list.append(v.split(':')[1])
    if(len(colored_node_list)>0): nx.draw_networkx_nodes(G,pos,vi_list,node_color=colors[i],node_size=550)

  
  for i in range(NUM_CLASSES):
    colored_edge_list=edges_by_color[i]
    print(colored_edge_list)
    if(len(colored_edge_list)>0):
      nx.draw_networkx_edges(G,pos,colored_edge_list,edge_color=colors[i])
  
  if(len(black_edges)>0):  nx.draw_networkx_edges(G,pos,black_edges,edge_color='black')

  nx.draw_networkx_labels(G,pos=pos,font_size=10)

  plt.show()
  
  graph_id_map[graph_id]=G
  
  
  graph_id+=1

  return G

In [ ]:
colors=['red','blue','green','yellow','pink','grey','orange','violet','brown','cyan','magenta','lime','silver']

In [ ]:
def create_map_from_list(list_obj):
  map_obj=dict()
  for i in range(len(list_obj)):
    map_obj[list_obj[i]]=i
  return map_obj

In [ ]:
color_map=create_map_from_list(colors)
print(color_map)

{'red': 0, 'blue': 1, 'green': 2, 'yellow': 3, 'pink': 4, 'grey': 5, 'orange': 6, 'violet': 7, 'brown': 8, 'cyan': 9, 'magenta': 10, 'lime': 11, 'silver': 12}


In [ ]:
#creating graphs from window segments where each graph contains maximum number of thresold nodes
def update_graphs(df,threshold):

  global graph_label_list
  df_list=df.groupby('user-id')
  users=df['user-id'].unique()
  for user in users:
    df_user=df_list.get_group(user)

    activities=df_user['encoded-activity'].unique()
    df_slice_list=df_user.groupby('encoded-activity')

    for activity in activities:

      df_slice=df_slice_list.get_group(activity)

      segments,labels,nodes=segment_labeling_overall(df_slice,TIME_STEPS,TIME_STEPS,FEATURES)

        
      for i in range (0,labels.shape[0],threeshold):
        print("---User#:"+str(user)+" Activity#:"+str(activity)+"-----")
        G=create_graph(labels[i:i+threeshold],segments[i:i+threeshold],colors)
        graph_label_list.append(activity)
      




  

In [ ]:
update_graphs(df)

Maximum no= 25


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 27
Done-->total27nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 9
Done-->total9nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 20
Done-->total20nodes formed
Maximum no= 22
Done-->total22nodes formed
Maximum no= 23
Done-->total23nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 25
Done-->total24nodes formed
Maximum no= 9
Done-->total8nodes formed
Maximum no=

In [ ]:
print(node_map)

{'1-red:0': 1, '1-red:1': 2, '1-red:2': 3, '1-red:3': 4, '1-red:4': 5, '1-red:5': 6, '1-red:6': 7, '1-red:7': 8, '1-red:8': 9, '1-red:9': 10, '1-red:10': 11, '1-red:11': 12, '1-red:12': 13, '1-red:13': 14, '1-red:14': 15, '1-red:15': 16, '1-red:16': 17, '1-red:17': 18, '1-red:18': 19, '1-red:19': 20, '1-red:20': 21, '1-red:21': 22, '1-red:22': 23, '2-blue:0': 24, '2-blue:1': 25, '2-blue:2': 26, '2-blue:3': 27, '2-blue:4': 28, '2-blue:5': 29, '2-blue:6': 30, '2-blue:7': 31, '2-blue:8': 32, '2-blue:9': 33, '2-blue:10': 34, '2-blue:11': 35, '2-blue:12': 36, '2-blue:13': 37, '2-blue:14': 38, '2-blue:15': 39, '2-blue:16': 40, '2-blue:17': 41, '2-blue:18': 42, '2-blue:19': 43, '2-blue:20': 44, '2-blue:21': 45, '2-blue:22': 46, '3-green:0': 47, '3-green:1': 48, '3-green:2': 49, '3-green:3': 50, '3-green:4': 51, '3-green:5': 52, '3-green:6': 53, '3-green:7': 54, '3-green:8': 55, '3-green:9': 56, '3-green:10': 57, '3-green:11': 58, '3-green:12': 59, '3-green:13': 60, '3-green:14': 61, '3-green:

In [ ]:
print(graph_id_map)

{1: <networkx.classes.graph.Graph object at 0x7fbe566f4438>, 2: <networkx.classes.graph.Graph object at 0x7fbe53d9cf98>, 3: <networkx.classes.graph.Graph object at 0x7fbe5665b908>, 4: <networkx.classes.graph.Graph object at 0x7fbe5380ca90>, 5: <networkx.classes.graph.Graph object at 0x7fbe51fa4240>, 6: <networkx.classes.graph.Graph object at 0x7fbe568e4c18>, 7: <networkx.classes.graph.Graph object at 0x7fbe57cb9e10>, 8: <networkx.classes.graph.Graph object at 0x7fbe568a3fd0>, 9: <networkx.classes.graph.Graph object at 0x7fbe582422b0>, 10: <networkx.classes.graph.Graph object at 0x7fbe57caf3c8>, 11: <networkx.classes.graph.Graph object at 0x7fbe51f2ef28>, 12: <networkx.classes.graph.Graph object at 0x7fbe568a3dd8>, 13: <networkx.classes.graph.Graph object at 0x7fbec31fa208>, 14: <networkx.classes.graph.Graph object at 0x7fbe53dca898>, 15: <networkx.classes.graph.Graph object at 0x7fbe51db28d0>, 16: <networkx.classes.graph.Graph object at 0x7fbe51c7e748>, 17: <networkx.classes.graph.Grap

In [ ]:
print(segments_map)

{'1-red:0': array([ 0.02301968, -0.88301395, -0.64456023,  0.21985768,  0.27213545,
        0.0640484 , -0.06885158, -0.06396234,  0.29145371,  0.15901754,
       -0.18789567, -0.18927512, -0.03088387,  0.10968385,  1.34717723,
       -0.49812507,  0.67354985, -0.1923307 , -0.38950022,  1.03076013,
        0.19603156,  0.02892326,  0.12396968]), '1-red:1': array([-0.15294295, -0.88346242, -0.65025331,  0.34236967,  0.54902983,
       -0.10870926,  0.1547863 ,  0.27119171,  0.26258662, -0.27127932,
        0.15715509,  0.10493746, -0.13479369, -0.51497882,  1.01763456,
       -0.14668062,  0.96403222, -0.44090391, -0.65265413,  1.36196905,
        0.11844807,  0.13787169,  0.0614807 ]), '1-red:2': array([-0.11461371, -0.76949925, -0.8207734 ,  0.85825924,  1.12768317,
        0.36014037,  0.19780276, -0.06463759,  0.50357309, -0.18782775,
       -0.22244466, -0.06246676, -0.01593148,  0.01560385,  1.2922788 ,
       -0.42297197,  0.39764918,  0.32815072, -0.43712855,  0.94486967,
      

In [ ]:
print(list_of_nodes)

['1-red:0', '1-red:1', '1-red:2', '1-red:3', '1-red:4', '1-red:5', '1-red:6', '1-red:7', '1-red:8', '1-red:9', '1-red:10', '1-red:11', '1-red:12', '1-red:13', '1-red:14', '1-red:15', '1-red:16', '1-red:17', '1-red:18', '1-red:19', '1-red:20', '1-red:21', '1-red:22', '2-blue:0', '2-blue:1', '2-blue:2', '2-blue:3', '2-blue:4', '2-blue:5', '2-blue:6', '2-blue:7', '2-blue:8', '2-blue:9', '2-blue:10', '2-blue:11', '2-blue:12', '2-blue:13', '2-blue:14', '2-blue:15', '2-blue:16', '2-blue:17', '2-blue:18', '2-blue:19', '2-blue:20', '2-blue:21', '2-blue:22', '3-green:0', '3-green:1', '3-green:2', '3-green:3', '3-green:4', '3-green:5', '3-green:6', '3-green:7', '3-green:8', '3-green:9', '3-green:10', '3-green:11', '3-green:12', '3-green:13', '3-green:14', '3-green:15', '3-green:16', '3-green:17', '3-green:18', '3-green:19', '3-green:20', '3-green:21', '3-green:22', '4-yellow:0', '4-yellow:1', '4-yellow:2', '4-yellow:3', '4-yellow:4', '4-yellow:5', '4-yellow:6', '4-yellow:7', '4-yellow:8', '4-yel

In [ ]:
print(list_of_edges)

[('1-red:0', '1-red:1'), ('1-red:1', '1-red:2'), ('1-red:2', '1-red:3'), ('1-red:3', '1-red:4'), ('1-red:4', '1-red:5'), ('1-red:5', '1-red:6'), ('1-red:6', '1-red:7'), ('1-red:7', '1-red:8'), ('1-red:8', '1-red:9'), ('1-red:9', '1-red:10'), ('1-red:10', '1-red:11'), ('1-red:11', '1-red:12'), ('1-red:12', '1-red:13'), ('1-red:13', '1-red:14'), ('1-red:14', '1-red:15'), ('1-red:15', '1-red:16'), ('1-red:16', '1-red:17'), ('1-red:17', '1-red:18'), ('1-red:18', '1-red:19'), ('1-red:19', '1-red:20'), ('1-red:20', '1-red:21'), ('1-red:21', '1-red:22'), ('2-blue:0', '2-blue:1'), ('2-blue:1', '2-blue:2'), ('2-blue:2', '2-blue:3'), ('2-blue:3', '2-blue:4'), ('2-blue:4', '2-blue:5'), ('2-blue:5', '2-blue:6'), ('2-blue:6', '2-blue:7'), ('2-blue:7', '2-blue:8'), ('2-blue:8', '2-blue:9'), ('2-blue:9', '2-blue:10'), ('2-blue:10', '2-blue:11'), ('2-blue:11', '2-blue:12'), ('2-blue:12', '2-blue:13'), ('2-blue:13', '2-blue:14'), ('2-blue:14', '2-blue:15'), ('2-blue:15', '2-blue:16'), ('2-blue:16', '2-

In [ ]:
print(node_label_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
print(graph_label_list)

[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4, 4, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4, 0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 8, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4, 4, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4, 4, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 0, 2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 4]


In [ ]:
print(len(node_map))
print(len(graph_id_map))
print(len(segments_map))
print(len(list_of_nodes))
print(len(list_of_edges))
print(len(node_label_list))
print(len(graph_label_list))

2574
120
2574
2574
2454
2574
120


In [ ]:
activity_map={}
activities=df['encoded-activity'].unique()
for activity in activities:
  name=df[df['encoded-activity']==activity]['activity'].unique()[0]
  activity_map[activity]=name

print(activity_map)

{0: 'Standing still', 1: 'Sitting and relaxing', 2: 'Lying down', 3: 'Walking', 5: 'Waist bends forward', 6: 'Frontal elevation of arms', 7: 'Knees bending', 8: 'Cycling', 9: 'Jogging', 10: 'Running', 11: 'Jump front & back', 4: 'Climbing stairs'}


In [ ]:
print(node_id,graph_id)

2575 121


In [ ]:
df_A=pd.DataFrame()
df_node_label=pd.DataFrame()
df_graph_label=pd.DataFrame()
df_node_attr=pd.DataFrame()
df_graph_indicator=pd.DataFrame()

In [ ]:
def create_node_dataframes(node_map,list_of_nodes,node_label_list,segments_map,activity_map):
  df_node_label=pd.DataFrame(columns=["label",'activity-name'])
  feature_col=[]
  for i in range(FEATURES):
    feature_col.append("mean_feature-"+str(i+1))
  df_node_attr=pd.DataFrame(columns=feature_col)
  df_graph_indicator=pd.DataFrame(columns=["graph-id"])
  cnt=len(node_label_list)
  print("Total node count--->"+str(cnt))
  for i in range(cnt):
    node=list_of_nodes[i]
    graph_id=node.split('-')[0]
    label=node_label_list[i]
    name=activity_map[label]
    segment=segments_map[node]
   
    df_node_label=df_node_label.append({"label":label+1,"activity-name":name},ignore_index=True)

    segment_dict={}
    for i in range(FEATURES):
      segment_dict[feature_col[i]]=segment[i]
    df_node_attr=df_node_attr.append(segment_dict,ignore_index=True)

    df_graph_indicator=df_graph_indicator.append({"graph-id":graph_id},ignore_index=True)
  
  df_node_label=df_node_label.astype({"label": int})
  df_graph_indicator=df_graph_indicator.astype({"graph-id": int})
  print("Dataframe creation done")
  return df_node_label,df_node_attr,df_graph_indicator

In [ ]:
df_node_label,df_node_attr,df_graph_indicator=create_node_dataframes(node_map,list_of_nodes,node_label_list,segments_map,activity_map)

Total node count--->2574
Dataframe creation done


In [ ]:
print(df_node_label)

      label    activity-name
0         1   Standing still
1         1   Standing still
2         1   Standing still
3         1   Standing still
4         1   Standing still
...     ...              ...
2569      5  Climbing stairs
2570      5  Climbing stairs
2571      5  Climbing stairs
2572      5  Climbing stairs
2573      5  Climbing stairs

[2574 rows x 2 columns]


In [ ]:
print(df_node_attr)

      mean_feature-1  mean_feature-2  ...  mean_feature-22  mean_feature-23
0           0.023020       -0.883014  ...         0.028923         0.123970
1          -0.152943       -0.883462  ...         0.137872         0.061481
2          -0.114614       -0.769499  ...        -0.150106        -0.109728
3          -0.167724       -0.465191  ...         0.159969         0.074742
4          -0.070746       -0.129681  ...        -0.009099         0.093226
...              ...             ...  ...              ...              ...
2569        0.067648       -0.072044  ...         0.184308         0.098243
2570       -0.011410       -0.173474  ...         0.511964         0.033352
2571        0.013603       -0.008870  ...        -0.367686        -0.021598
2572       -0.003885        0.579825  ...        -0.471345         0.038043
2573       -0.091757       -0.129022  ...         0.229377        -0.131399

[2574 rows x 23 columns]


In [ ]:
print(df_graph_indicator)

      graph-id
0            1
1            1
2            1
3            1
4            1
...        ...
2569       120
2570       120
2571       120
2572       120
2573       120

[2574 rows x 1 columns]


In [ ]:
def create_edge_dataframe(edge_list,node_map):
  df=pd.DataFrame(columns=["node-1","node-2"])
  cnt=len(edge_list)
  print("Total edge count-->"+str(cnt))
  #id=1

  for i in range(cnt):
    edge=edge_list[i]
    node1=edge[0]
    node2=edge[1]
    print("node-1-->"+node1)
    print("node-2-->"+node2)
    id1=node_map[node1]
    id2=node_map[node2]
    df=df.append({"node-1":id1,"node-2":id2},ignore_index=True)
    df=df.append({"node-1":id2,"node-2":id1},ignore_index=True)
    
  df=df.astype({"node-1": int, "node-2": int})
    # id+=1
  
  return df

In [ ]:
df_A=create_edge_dataframe(list_of_edges,node_map)

Total edge count-->2454
node-1-->1-red:0
node-2-->1-red:1
node-1-->1-red:1
node-2-->1-red:2
node-1-->1-red:2
node-2-->1-red:3
node-1-->1-red:3
node-2-->1-red:4
node-1-->1-red:4
node-2-->1-red:5
node-1-->1-red:5
node-2-->1-red:6
node-1-->1-red:6
node-2-->1-red:7
node-1-->1-red:7
node-2-->1-red:8
node-1-->1-red:8
node-2-->1-red:9
node-1-->1-red:9
node-2-->1-red:10
node-1-->1-red:10
node-2-->1-red:11
node-1-->1-red:11
node-2-->1-red:12
node-1-->1-red:12
node-2-->1-red:13
node-1-->1-red:13
node-2-->1-red:14
node-1-->1-red:14
node-2-->1-red:15
node-1-->1-red:15
node-2-->1-red:16
node-1-->1-red:16
node-2-->1-red:17
node-1-->1-red:17
node-2-->1-red:18
node-1-->1-red:18
node-2-->1-red:19
node-1-->1-red:19
node-2-->1-red:20
node-1-->1-red:20
node-2-->1-red:21
node-1-->1-red:21
node-2-->1-red:22
node-1-->2-blue:0
node-2-->2-blue:1
node-1-->2-blue:1
node-2-->2-blue:2
node-1-->2-blue:2
node-2-->2-blue:3
node-1-->2-blue:3
node-2-->2-blue:4
node-1-->2-blue:4
node-2-->2-blue:5
node-1-->2-blue:5
node-

In [ ]:
print(df_A)

      node-1  node-2
0          1       2
1          2       1
2          2       3
3          3       2
4          3       4
...      ...     ...
4903    2572    2571
4904    2572    2573
4905    2573    2572
4906    2573    2574
4907    2574    2573

[4908 rows x 2 columns]


In [ ]:
def create_graph_dataframe(graph_label_list,activity_map):
  df=pd.DataFrame(columns=["Label","Activity-Name"])
  cnt=len(graph_label_list)
  print("No of graphs--->"+str(cnt))
  for i in range(cnt):
    id=i+1
    label=graph_label_list[i]
    name=activity_map[label]
    df=df.append({"Label": label+1,"Activity-Name": name},ignore_index=True)
  df=df.astype({"Label": int})
  return df

In [ ]:
df_graph_label=create_graph_dataframe(graph_label_list,activity_map)
print(df_graph_label)

No of graphs--->120
     Label         Activity-Name
0        1        Standing still
1        2  Sitting and relaxing
2        3            Lying down
3        4               Walking
4        6   Waist bends forward
..     ...                   ...
115      9               Cycling
116     10               Jogging
117     11               Running
118     12     Jump front & back
119      5       Climbing stairs

[120 rows x 2 columns]


In [ ]:
def save_dataframe_to_txt(df,filepath):
  df.to_csv(filepath,header=None,index=None,sep=',',mode='w')

In [ ]:
sourcepath='/content/drive/My Drive/MHEALTH_Graph_Classification/raw'

In [ ]:
save_dataframe_to_txt(df_A,sourcepath+'/MHEALTH_A.txt')
save_dataframe_to_txt(df_graph_indicator,sourcepath+'/MHEALTH_graph_indicator.txt')
save_dataframe_to_txt(df_graph_label,sourcepath+'/MHEALTH_graph_label.txt')
save_dataframe_to_txt(df_node_attr,sourcepath+'/MHEALTH_node_attr.txt')
save_dataframe_to_txt(df_node_label,sourcepath+'/MHEALTH_node_label.txt')